In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from ipynb.fs.full.utility_functions import gen_data_dict, get_data, get_cumsum, error_count, write_to_csv, opart, SquaredHingeLoss

np.random.seed(4)
torch.manual_seed(4)

In [2]:
# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=8, num_layers=1, nonlinearity='relu', bias=True, batch_first=False)
        self.fc1 = nn.Linear(8, 1)

    def forward(self, x):
        _, x = self.rnn(x)
        x    = self.fc1(x)
        return x

In [3]:
# Load model1
model1 = RNNModel()
model1.load_state_dict(torch.load('saved_models/model1_rnn_relu_best.pth'))
model1.eval()  # Set the model to evaluation mode

# Load model2
model2 = RNNModel()
model2.load_state_dict(torch.load('saved_models/model2_rnn_relu_best.pth'))
model2.eval()  # Set the model to evaluation mode

RNNModel(
  (rnn): RNN(1, 8)
  (fc1): Linear(in_features=8, out_features=1, bias=True)
)

In [4]:
seqs   = gen_data_dict('sequence_label_data/signals.gz')
labels = gen_data_dict('sequence_label_data/labels.gz')

In [5]:
ldas1 = np.zeros(len(seqs))
ldas2 = np.zeros(len(seqs))
with torch.no_grad():
    for i in range(len(seqs)):
        seq = torch.tensor(seqs[i][1]['logratio'].to_numpy(), dtype=torch.float32).reshape(-1,1)
        ldas1[i] = model1(seq).numpy()[0][0]
        ldas2[i] = model2(seq).numpy()[0][0]

In [ ]:
header = ['sequenceID', 'fold_1_total_labels', 'fold_2_total_labels', 'fold_1_errs', 'fold_2_errs']

for i in range(len(seqs)):
    # generate data
    sequence, neg_start_1, neg_end_1, pos_start_1, pos_end_1, neg_start_2, neg_end_2, pos_start_2, pos_end_2 = get_data(i, seqs=seqs, labels=labels)
    sequence_length = len(sequence)-1

    # vectors of cumulative sums
    y, z = get_cumsum(sequence)

    # get total labels
    fold1_total_labels = len(neg_start_1) + len(pos_start_1)
    fold2_total_labels = len(neg_start_2) + len(pos_start_2)

    # run each lambda and record it into csv file
    row  = [i, fold1_total_labels, fold2_total_labels]

    chpnt_fold1 = opart(10**ldas2[i], sequence)
    chpnt_fold2 = opart(10**ldas1[i], sequence)

    err_1 = error_count(chpnt_fold1, neg_start_1, neg_end_1, pos_start_1, pos_end_1)
    err_2 = error_count(chpnt_fold2, neg_start_2, neg_end_2, pos_start_2, pos_end_2)
    
    row.append(sum(err_1))
    row.append(sum(err_2))

    write_to_csv('learning_output/rnn_relu.csv', header, row)